In [2]:
import os
from typing import List, Tuple, Optional
from openai import OpenAI
from transformers import AutoTokenizer
from tqdm import tqdm
# load doc
with open("/root/app/summary/santi2.txt", "r") as file:
    santi2 = file.read()

# 打印前1000个字符
print(santi2[:1000])

《三体II：黑暗森林》
作者：刘慈欣

前言

序章

　　褐蚁已经忘记这里曾是它的家园。这段时光对于暮色中的大地和刚刚出现的星星来说短得可以忽略不计，但对于它来说却是漫长的。

　　在那个已被忘却的日子里，它的世界颠覆了。泥土飞走，出现了一条又深又宽的峡谷，然后泥土又轰隆隆地飞回来，峡谷消失了，在原来峡谷的尽头出现了一座黑色的孤峰。其实，在这片广阔的疆域上，这种事常常发生，泥土飞走又飞回，峡谷出现又消失，然后是孤峰降临，好像是给每次灾变打上一个醒目的标记。褐蚁和几百个同族带着幸存的蚁后向太阳落下的方向走了一段路，建立了新的帝国。

　　这次褐蚁来到故地，只是觅食途中偶然路过而已。它来到孤峰脚下，用触须摸了摸这顶天立地的存在，发现孤峰的表面坚硬光滑，但能爬上去，于是它向上爬去。没有什么且的，只是那小小的简陋神经网络中的一次随机扰动所致。这扰动随处可见，在地面的每一株小草和草叶上的每一粒露珠中，在天空中的每一片云和云后的每一颗星辰上……扰动都是无目的的，但巨量的无目的扰动汇集在一起，目的就出现了。

　　褐蚁感到了地面的震动，从震动由弱变强的趋势来判断，它知道地面上的另一个巨大的存在正在向这里运动，它没有理会，继续向孤峰上攀爬。在孤峰底部和地面形成的直角空间里有一面蛛网，褐蚁知道那是什么，它小心地绕过了粘在悬崖上的蛛丝，从那个缩起所有的腿静等着蛛丝震动的蜘蛛旁经过，它们彼此都感觉到了对方的存在，但同过去的一亿年一样，双方没有任何交流。

　　震动达到高峰后停止了，那个巨大的存在已经来到了孤峰前，褐蚁看到这个存在比孤峰还要高许多，遮住了很大一部分天空。对这类存在褐蚁并不陌生，它知道他们是活的，常常出现在这片疆域，那些出现后很快就消失的峡谷和越来越多地耸现的孤峰，都与他们有着密切的关系。

　　褐蚁继续向上攀登，它知道这类存在一般不会威胁到自己——当然也有例外。对于已处于下方的那个蜘蛛，这种例外已经出现，那个存在显然发现了孤峰与地面之间的蛛网，用一个肢体上拿着的一束花的花柄拂去了它，蜘蛛随着断开的蛛丝落到了草丛中。然后，他把花轻轻地放在了孤峰前。

　　这时。另一个震动出现了，很微弱，但也在增强中。褐蚁知道，另一个同类型的存在正在向孤峰移动。与此同时，在前方的峭壁上，它遇到了一道长长的沟槽，与峭壁表面相比，沟槽的凹面粗糙一些，颜色也不同，呈灰白色，它沿着沟槽爬，粗糙的表面

In [3]:
# 使用 tiktoken 模块获取编码器
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

# 编码文本并打印编码后的长度
len(encoding.encode(santi2))
# 403661 四十万多个字符

403661

In [4]:
client = OpenAI(
    base_url = "http://10.176.50.47:34067/v1",
    api_key='ollama', # required, but unused
)

def get_chat_completion(messages, model="/root/app/models/Qwen1.5-32B-Chat/"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [5]:
messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": '你好吗？'},
        ]
res = get_chat_completion(messages=messages)
print(res)

你好！我是一个计算机程序，所以我不能感受到身体上的感觉。但是我随时准备为你提供帮助。你有什么问题吗？


In [7]:
# 对文本进行 embedding
def tokenize(text: str) -> List[str]:    
    return encoding.encode(text)

# 根据最大 token 数和分隔符将文本分成较小的片段。
def chunk_on_delimiter(input_string: str,
                       max_tokens: int, 
                       delimiter: str) -> List[str]:
    # 按分隔符拆分输入
    chunks = input_string.split(delimiter)

    combined_chunks, _, dropped_chunk_count = combine_chunks_with_no_minimum(
        chunks, 
        max_tokens, 
        chunk_delimiter=delimiter, 
        add_ellipsis_for_overflow=True
    )
    if dropped_chunk_count > 0:
        print(f"warning: {dropped_chunk_count} chunks were dropped due to overflow")
    combined_chunks = [f"{chunk}{delimiter}" for chunk in combined_chunks]
    return combined_chunks


# 将文本块组合成更大的块，而不超过模型的最大窗口。返回组合的文本块、它们的原始索引以及由于溢出而丢弃的块的计数。
def combine_chunks_with_no_minimum(
        chunks: List[str],
        max_tokens: int,
        chunk_delimiter="\n\n",
        header: Optional[str] = None,
        add_ellipsis_for_overflow=False,
) -> Tuple[List[str], List[int]]:
    dropped_chunk_count = 0
    output = []  # list to hold the final combined chunks 包含最终组合块的列表
    output_indices = []  # list to hold the indices of the final combined chunks 最终组合块的索引
    candidate = ([] if header is None else [header])  # list to hold the current combined chunk candidate 当前组合块候选的列表
    candidate_indices = []  # list to hold the indices of the current combined chunk candidate 当前组合块候选的索引
    for chunk_i, chunk in enumerate(chunks):
        chunk_with_header = [chunk] if header is None else [header, chunk]
        if len(tokenize(chunk_delimiter.join(chunk_with_header))) > max_tokens:
            print(f"warning: chunk overflow")
            if (
                    add_ellipsis_for_overflow
                    and len(tokenize(chunk_delimiter.join(candidate + ["..."]))) <= max_tokens
            ): 
                candidate.append("...") # add ellipsis to the candidate if it fits
                dropped_chunk_count += 1
            continue  # this case would break downstream assumptions
        
        # estimate token count with the current chunk added
        extended_candidate_token_count = len(tokenize(chunk_delimiter.join(candidate + [chunk])))
        # If the token count exceeds max_tokens, add the current candidate to output and start a new candidate
        if extended_candidate_token_count > max_tokens:
            output.append(chunk_delimiter.join(candidate))
            output_indices.append(candidate_indices)
            candidate = chunk_with_header  # re-initialize candidate
            candidate_indices = [chunk_i]
        # otherwise keep extending the candidate
        else:
            candidate.append(chunk)
            candidate_indices.append(chunk_i)
    # add the remaining candidate to output if it's not empty
    if (header is not None and len(candidate) > 1) or (header is None and len(candidate) > 0):
        output.append(chunk_delimiter.join(candidate))
        output_indices.append(candidate_indices)
    return output, output_indices, dropped_chunk_count

In [8]:
def summarize(text: str,
              detail: float = 0,
              model: str = 'claude3.5-sonnet',
              additional_instructions: Optional[str] = None,
              minimum_chunk_size: Optional[int] = 512,
              chunk_delimiter: str = "\n",
              summarize_recursively=False,
              verbose=False):
    """
    总结给定的文本，方法是将其分成块，每个块都被单独总结。 摘要中的详细程度可以调整，该过程可以选择递归进行。
    
    Parameters:
    - text (str): The text to be summarized.
    - detail (float, optional): 在摘要中所需的详细程度的0到1之间的值。0导致更高级别的摘要，1导致更详细的摘要。默认为0。
    - model (str, optional): The model to use for generating summaries. Defaults to 'claude3.5-sonnet'.
    - additional_instructions (Optional[str], optional): 额外的说明，以便为自定义摘要提供给模型。默认为None。
    - minimum_chunk_size (Optional[int], optional): The minimum size for text chunks. Defaults to 512.
    - chunk_delimiter (str, optional):  用于将文本拆分为块的分隔符。默认为“\n”。
    - summarize_recursively (bool, optional):  如果为True，则使用先前的摘要作为上下文递归生成摘要。默认为False。
    - verbose (bool, optional):  是否打印有关分块过程的详细信息。默认为False。

    Returns:
    - str:  文本的最终编译摘要。
    
    首先，函数根据detail参数在最小和最大分块数量之间进行插值，以确定分块的数量。
    然后，它将文本分割成多个分块，并对每个分块进行摘要。如果summarize_recursively为True，则每个摘要都是基于之前的摘要生成的，
    这为摘要过程增加了更多的上下文信息。函数最终返回所有分块的汇总摘要。
    """

    # check detail is set correctly
    assert 0 <= detail <= 1
    # interpolate the number of chunks based to get specified level of detail
    max_chunks = len(chunk_on_delimiter(text, minimum_chunk_size, chunk_delimiter))
    min_chunks = 1
    num_chunks = int(min_chunks + detail * (max_chunks - min_chunks))

    # adjust chunk_size based on interpolated number of chunks
    document_length = len(tokenize(text))
    chunk_size = max(minimum_chunk_size, document_length // num_chunks)
    text_chunks = chunk_on_delimiter(text, chunk_size, chunk_delimiter)
    if verbose:
        print(f"Splitting the text into {len(text_chunks)} chunks to be summarized.")
        print(f"Chunk lengths are {[len(tokenize(x)) for x in text_chunks]}")

    # set system message
    system_message_content = "Rewrite this text in summarized form."
    if additional_instructions is not None:
        system_message_content += f"\n\n{additional_instructions}"

    accumulated_summaries = []
    for chunk in tqdm(text_chunks):
        if summarize_recursively and accumulated_summaries:
            # Creating a structured prompt for recursive summarization
            accumulated_summaries_string = '\n\n'.join(accumulated_summaries)
            user_message_content = f"Previous summaries:\n\n{accumulated_summaries_string}\n\nText to summarize next:\n\n{chunk}"
        else:
            # Directly passing the chunk for summarization without recursive context
            user_message_content = chunk

        # Constructing messages based on whether recursive summarization is applied
        messages = [
            {"role": "system", "content": system_message_content},
            {"role": "user", "content": user_message_content}
        ]

        # Assuming this function gets the completion and works as expected
        response = get_chat_completion(messages)
        accumulated_summaries.append(response)

    # Compile final summary from partial summaries
    final_summary = '\n\n'.join(accumulated_summaries)

    return final_summary

In [12]:
summary_with_additional_instructions = summarize(santi2, detail=0.1, minimum_chunk_size=1024, summarize_recursively=False, verbose=True,
                                                 additional_instructions="Write in point form and focus on numerical data.")
print(summary_with_additional_instructions)

Splitting the text into 44 chunks to be summarized.
Chunk lengths are [9260, 9381, 9380, 9382, 9251, 9141, 9383, 9370, 9279, 9355, 9143, 9363, 9335, 9309, 9359, 9233, 9300, 9293, 9361, 9364, 9340, 9327, 9250, 9326, 9342, 9386, 9348, 9287, 9140, 9387, 9327, 9336, 8945, 9192, 9304, 9266, 9258, 9363, 9059, 9284, 9351, 9365, 9271, 3965]


100%|██████████| 44/44 [09:08<00:00, 12.46s/it]

- 《三体II：黑暗森林》是刘慈欣的作品。
- 故事中，一只褐蚁在孤峰上攀爬，遇到不同形状的凹槽，对美感和方向有原始的感知。
- 两个活着的存在，叶老师和罗辑，对话中提到了宇宙社会学，一个假设中研究宇宙中众多文明的学科。
- 叶老师提出宇宙社会学的两个公理：生存是文明的第一需要，文明不断增长和扩张，但宇宙中的物质总量保持不变。
- 叶老师还提及猜疑链和技术爆炸两个概念，并建议罗辑创立这门学科。
- 罗辑对宇宙社会学表示兴趣，但自知无法成为像欧几里得那样的理论家。
- 对话中，叶文洁和罗辑讨论了人类文献资料理解困难的问题，发现关键在于同义词“想”和“说”的理解差异。
- 伊文斯与三体世界的智子交流，发现三体文明的思维对外界完全暴露，没有欺骗和谎言的概念。
- 伊文斯意识到这使得三体文明无法进行复杂的战略思维。
- 伊文斯认为人类的交流器官是进化缺陷，但三体文明认为这是效率更高的交流方式。
- 三体世界的信息交流中断，伊文斯感到时间的流逝。

- 吴岳和章北海在“唐”号航母的第四次近海编队训练中担任尴尬角色，因为“唐”号还在建造中，由“郑和”号代替。
- 章北海的父亲曾是他们的上级，吴岳曾就沟通问题与其交谈，将军建议只需搞好工作，无需了解思维方式。
- “唐”号看起来古老而沧桑，吴岳和章北海意识到可能无法按计划进行第五次编队训练。
- 在阿拉斯加格里利堡，北美防空司令部的NMD系统错误地拦截了国际空间站废弃反射膜，被误认为是敌方目标。
- NMD系统控制室的人员对错误感到懊恼，他们本想记录下可能是人类首次攻击敌人的历史时刻。
- 张援朝退休后，对世界局势的关注让他意识到邻居杨晋文关于关心国家大事的建议是正确的。
- 张援朝在电视新闻中得知NMD成功摧毁了低维展开的智子，以及贾维彬院士的死因并非智子攻击。
- 邻居苗福全因矿上问题需要处理而返回，感叹即将来临的艰难时期。
- 破壁人二号在准备自杀时，智子通过字幕与他交流，通过一些细节证实其身份，并告知一些破壁人未知的信息，破壁人二号因此确认了智子的存在并感到宽慰。

- 破壁人二号和字幕的对话揭示了双方在交流方式和欺骗手段上的差异。人类的欺骗和计谋虽然简陋，但智子的存在使人类世界对三体透明，除了思维。
- 字幕解释了三体社会的交流更倾向于思维透明，导致其文学作品难以理解。ETO（地球三体组织）因被三体抛弃而面临精神上的打击